In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd

service = Service(executable_path='/Users/kater/Downloads/chromedriver-mac-x64/chromedriver')
driver = webdriver.Chrome(service=service)

def fetch_setlist_data(url):
    driver.get(url)
    time.sleep(5)  

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    artist_element = soup.find(class_='setlistHeadline')
    artist = artist_element.text.replace('Setlist', '').strip() if artist_element else "Artist not found"

    date_location_element = soup.find('div', class_='dateBlock')
    date_location = ' '.join(date_location_element.text.split()) if date_location_element else "Date and location not found"

    location_name = soup.select_one('a[title^="More setlists from"] > span').text.strip()
    

    start_time_element = soup.find('div', class_='mainTime', string=lambda t: "PM" in t or "AM" in t)
    start_time = start_time_element.text.strip() if start_time_element else "Start time not found"
    

    tour_avg_duration_element = soup.find('div', class_='mainTime', string=lambda t: "h" in t and "m" in t)
    tour_avg_duration = tour_avg_duration_element.text.strip() if tour_avg_duration_element else "Tour duration not found"

    songs_elements = soup.find('ol', class_='songsList')
    songs = []
    if songs_elements:
        for li in songs_elements.find_all('li', recursive=False):
            song_part = li.find('div', class_='songPart')
            if song_part:
                song_title = song_part.text.strip()
                songs.append(song_title)
            else:
                tape_part = li.find('div', class_='songTooltip')
                if tape_part:
                    tape_title = tape_part.text.strip()
                    songs.append(tape_title)

    data = {
        'Artist': artist.split('\n')[0],
        'Date and Location': date_location,
        'Location Name': location_name,
        'Start Time': start_time,
        'Tour Average Duration': tour_avg_duration,
        'Songs': songs,
    }
    
    return pd.DataFrame([data])



url = 'https://www.setlist.fm/setlist/morgan-wallen/2024/lucas-oil-stadium-indianapolis-in-5bab1778.html'
data_frame = fetch_setlist_data(url)
print(data_frame)

driver.quit()


exploded_df = data_frame.explode('Songs')

exploded_df.to_csv('setlist_data.csv', index=False)

print("CSV file has been updated with the latest detailed data.")



           Artist Date and Location                             Location Name  \
0  Morgan Wallen         Apr 5 2024  Lucas Oil Stadium, Indianapolis, IN, USA   

  Start Time Tour Average Duration  \
0    9:30 PM                 4h 4m   

                                               Songs  
0  [Broadway Girls, Ain't That Some, I Wrote the ...  
CSV file has been updated with the latest detailed data.
